In [1]:
# | default_exp routes.jupyter

In [2]:
# | exporti
import httpx

import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.get_data as gd

import domolibrary.client.DomoError as de
import domolibrary.utils.chunk_execution as ce

import os

[![Tutorial Video pt 2 of 4]](https://youtu.be/7aTHw7scsa8)

### GET Routes


In [3]:
#| export

class JupyterApi_Error(de.DomoError):
    def __init__(self, status, response, domo_instance,  parent_class = None, function_name = None):
        super().__init__(
            status=status,
            message = response,
            domo_instance = domo_instance,
            parent_class = parent_class,
            function_name = function_name
        )

In [4]:
#| export

@gd.route_function
async def get_jupyter_workspaces(
    auth :dmda.DomoAuth,
    parent_class: str = None,
    session : httpx.AsyncClient = None,
    debug_num_stacks_to_drop=1,
    debug_api: bool = False,
    debug_loop : bool = False
):
    url = f"https://{auth.domo_instance}.domo.com/api/datascience/v1/search/workspaces"

    body = {"limit":50,"offset":0,"sortFieldMap":{"LAST_RUN":"DESC"},"filters":[]}

    def arr_fn(res):
        return res.response['workspaces']

    offset_params = {
        "limit" : "limit",
        "offset" : "offset"
    }

    res = await gd.looper(
        url=url,
        method="POST",
        limit = 50,
        body = body,
        auth=auth,
        arr_fn = arr_fn, 
        offset_params_in_body=True,
        offset_params = offset_params,
        parent_class=parent_class,
        session = session,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        debug_api=debug_api,
        debug_loop = debug_loop,
    )
    if not res.is_success:
        raise JupyterApi_Error(status = res.status, response= res.response, domo_instance = auth.domo_instance, parent_class = parent_class, function_name = res.traceback_details.function_name)

    return res

In [5]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res_domo_workspaces = await get_jupyter_workspaces(token_auth, debug_loop = False, debug_api = False)
res_domo_workspaces

ConnectTimeout: 

In [ ]:
[res['name'] for res in res_domo_workspaces.response]

['abcdef',
 'learn_jupyter',
 'Jupyter Test',
 't1',
 'monit',
 'The Office',
 'Elliott Leonard Test',
 'jw',
 'Oleksii - workspace',
 'test_asm',
 'Aditya Jain WS',
 "Jason's Jeans Python Space",
 'VS | DP21 Workshop',
 'Tutorial',
 'KL | DP21 Workshop ',
 'Test',
 'Prueba']

In [ ]:
# | export


@gd.route_function
async def start_jupyter_workspace(
    workspace_id,
    auth: dmda.DomoAuth,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop=1,
    debug_api: bool = False,
):
    url = f"https://{auth.domo_instance}.domo.com/api/datascience/v1/workspaces/{workspace_id}/instances"
    # url = f"https://domo-community.domo.com/api/datascience/v1/workspaces/1cfe9db4-5937-4889-beb3-a311fc42f246/instances"

    try:
        res = await gd.get_data(
            url=url,
            method="POST",
            auth=auth,
            parent_class=parent_class,
            session=session,
            num_stacks_to_drop=debug_num_stacks_to_drop,
            debug_api=debug_api,
        )
    except httpx.HTTPError as e:
        return rgd.ResponseGetData(
            status=500,
            response=f"starting workspace, please wait - {e}",
            is_success=False,
        )

    if res.status == 500 or res.status == 403:
        raise JupyterApi_Error(
            status=res.status,
            response=f"you may not have access to this workspace {workspace_id}, is it shared with you? -- {res.response}",
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    if not res.is_success:
        raise JupyterApi_Error(
            status=res.status,
            response=res.response,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    res.response = "workspace started"
    return res

In [ ]:
test_workspace = res_domo_workspaces.response[4]
print(test_workspace['name'])
# test_workspace
await start_jupyter_workspace(workspace_id = test_workspace['id'], auth = token_auth, debug_api = True)


NameError: name 'res_domo_workspaces' is not defined

In [ ]:
# | export

@gd.route_function
async def get_jupyter_content(
    auth: dmda.DomoAuth,
    content_path: str = "",
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class: str = None,
    session : httpx.AsyncClient = None
):
    url = f"https://{auth.domo_instance}.{auth.service_location}{auth.service_prefix}api/contents/{content_path}"

    res = await gd.get_data(
        url=f"{url}",
        method="GET",
        auth=auth,
        headers={"authorization": f"Token {auth.jupyter_token}"},
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        session = session
    )
    if not res.is_success:
        raise JupyterApi_Error(
            status=res.status,
            response=res.response,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

#### sample implementation of get_jupyter_content


In [ ]:
# | eval: false

import os
import domolibrary.client.DomoAuth as dmda

jupyter_token = "mgaHHqXQafZhWMIcbT5cVsW9KZY7c4"
service_location = "jupyter-prod1.domodatascience.com"
service_prefix = "/user/domo-community-1893952720/19fb3535/"

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


dj_auth = dmda.DomoJupyterTokenAuth.convert_auth(
    token_auth=token_auth,
    jupyter_token=jupyter_token,
    service_location=service_location,
    # service_prefix=None
    service_prefix=service_prefix,
)

res = await get_jupyter_content(
    auth=dj_auth,
    # content_path = 'my_terrible_demo.txt',
    # content_path="datatypes.ipynb",
    debug_api=None,
)

res.response["content"][0:2]

[{'name': 'my_terrible_demo.txt',
  'path': 'my_terrible_demo.txt',
  'last_modified': '2023-09-15T22:20:36.711000Z',
  'created': '2023-09-15T22:20:36.711000Z',
  'content': None,
  'format': None,
  'mimetype': 'text/plain',
  'size': 33,
  'writable': True,
  'type': 'file'},
 {'name': 'untitled1.txt',
  'path': 'untitled1.txt',
  'last_modified': '2023-09-15T22:22:55.746000Z',
  'created': '2023-09-15T22:22:55.746000Z',
  'content': None,
  'format': None,
  'mimetype': 'text/plain',
  'size': 33,
  'writable': True,
  'type': 'file'}]

### update jupyter content

improve code maintenance by implementing a factory design patternx


In [ ]:
# | exporti
def generate_update_jupyter_text(body):
    body.update(
        {
            "format": "text",
            "type": "file",
        }
    )
    return body


def generate_update_jupyter_ipynb(body):
    body.update(
        {
            "format": None,
            "type": "notebook",
        }
    )
    return body


def generate_update_jupyter_directory(content_path, body):
    if "/" in content_path:
        new_content_path = "/".join(content_path.split("/")[:-1])
    else:
        new_content_path = ""

    body.update(
        {
            "path": new_content_path,
            "format": None,
            "type": "directory",
        }
    )
    return body

In [ ]:
# |export


def generate_update_jupyter_body(
    new_content, content_path: str  # my_folder/datatypes.ipynb
):
    """factory to construct properly formed body"""

    content_name = os.path.normpath(content_path).split(os.sep)[-1]

    if "." in content_path:
        content_type = content_path.split(".")[-1]
    else:
        content_type = "directory"

    body = {
        "name": content_name,
        "content": new_content,
        "path": content_path,
    }

    if content_type == "ipynb":
        return generate_update_jupyter_ipynb(body)

    if content_type == "directory":
        return generate_update_jupyter_directory(content_path, body)

    return generate_update_jupyter_text(body)


generate_update_jupyter_body("hello world", "hi.md")

{'name': 'hi.md',
 'content': 'hello world',
 'path': 'hi.md',
 'format': 'text',
 'type': 'file'}

## CRUD Routes


In [ ]:
# | export

async def update_jupyter_file(
    auth: dmda.DomoJupyterAuth,
    new_content,
    content_path: str = "",  # file name and location in jupyter
    debug_api: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop=1,
):
    dmda.test_is_jupyter_auth(auth)

    body = generate_update_jupyter_body(new_content, content_path)

    content_path_split = os.path.normpath(content_path).split(os.sep)

    url = f"https://{auth.domo_instance}.{auth.service_location}{auth.service_prefix}api/contents/{'/'.join(content_path_split)}"

    res = await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        body=body,
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if not res.is_success:
        raise JupyterApi_Error(
            status=res.status,
            response=res.response,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

#### sample implementation of update_content


In [ ]:
# |eval: false

import os
import domolibrary.client.DomoAuth as dmda

jupyter_token = "mgaHHqXQafZhWMIcbT5cVsW9KZY7c4"
service_location = "jupyter-prod1.domodatascience.com"
service_prefix = "/user/domo-community-1893952720/19fb3535/"

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

dj_auth = dmda.DomoJupyterTokenAuth.convert_auth(
    token_auth=token_auth,
    jupyter_token=jupyter_token,
    service_location=service_location,
    # service_prefix=None
    service_prefix=service_prefix,
)

res = await update_jupyter_file(
    auth=dj_auth,
    content_path="new_folder/my_great_demo.txt",
    new_content="jae is excellent at demoes",
    debug_api=False,
)

res.response

{'name': 'my_great_demo.txt',
 'path': 'new_folder/my_great_demo.txt',
 'last_modified': '2023-09-15T22:24:35.531000Z',
 'created': '2023-09-15T22:24:35.531000Z',
 'content': None,
 'format': None,
 'mimetype': 'text/plain',
 'size': 26,
 'writable': True,
 'type': 'file'}

### recursive get_content

[![Tutorial Video pt 3 of 4]](https://youtu.be/l32qTj08bCY)


In [ ]:
# | exporti


async def get_content_recursive_process_obj(
    obj,
    all_rows,
    auth,
    debug_api: bool = False,
    debug_num_stacks_to_drop=0,
    parent_class=None,
):
    content_path = obj["path"]

    if content_path.startswith("recent_executions"):
        return

    if obj["type"] != "directory":
        res = await get_jupyter_content(
            auth=auth,
            content_path=content_path,
            debug_api=debug_api,
            parent_class=parent_class,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop + 1,
        )

        all_rows.append(res.response)

    elif obj["type"] == "directory":
        await get_content_recursive(
            auth=auth,
            content_path=content_path,
            all_rows=all_rows,
            debug_api=debug_api,
            parent_class=parent_class,
            debug_num_stacks_to_drop=debug_num_stacks_to_drop + 1,
        )

In [ ]:
# | export


async def get_content_recursive(
    auth: dmda.DomoJupyterAuth,
    content_path="",
    all_rows=None,
    debug_api: bool = False,
    return_raw: bool = False,
    is_skip_recent_executions: bool = True,
    debug_num_stacks_to_drop=2,
    parent_class: str = None,
):
    dmda.test_is_jupyter_auth(auth)

    all_rows = all_rows or []

    res = await get_jupyter_content(
        auth=auth,
        content_path=content_path,
        debug_api=debug_api,
        parent_class=parent_class,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    content_ls = res.response["content"]

    await ce.gather_with_concurrency(
        n=5,
        *[
            get_content_recursive_process_obj(
                obj,
                all_rows,
                auth,
                debug_api=debug_api,
                parent_class=parent_class,
                debug_num_stacks_to_drop=debug_num_stacks_to_drop,
            )
            for index, obj in enumerate(content_ls)
        ]
    )

    if return_raw:
        return res

    res.response = all_rows

    return res

#### sample implementation of get_content_recursive


In [ ]:
# |eval: false

import os
import domolibrary.client.DomoAuth as dmda

jupyter_token = "mgaHHqXQafZhWMIcbT5cVsW9KZY7c4"
service_location = "jupyter-prod1.domodatascience.com"
service_prefix = "/user/domo-community-1893952720/19fb3535/"


dj_auth = dmda.DomoJupyterTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    jupyter_token=jupyter_token,
    service_location=service_location,
    # service_prefix=None
    service_prefix=service_prefix,
)

res = await get_content_recursive(auth=dj_auth, content_path="", debug_api=False)

[content["name"] for content in res.response]

['untitled1.txt',
 'my_terrible_demo.txt',
 'tutorial_env.ipynb',
 'UpdateDatasets.ipynb',
 'MakeDomoFaster.ipynb',
 'get_content.ipynb',
 'untitled1.txt',
 'DataflowExecution.ipynb',
 'Untitled.ipynb',
 'instance_pdp_access.ipynb',
 'README.md',
 'untitled.txt',
 'env.txt',
 'my_great_demo.txt',
 'datatypes.ipynb']

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()